introduces a language called Pig Latin, that is a lot like SQL.
translate into MapReduce and do things more quickly
can also run top of Tez, directed acyclic graph, find interdepencies and run faster

Grunt: run each line
Script
Ambari/Hue

if execute on Tez, much quicker

syntax:
LOAD STORE DUMP
    STORE ratings INTO 'ourRatings' using PigStorage(':');
FILTER DISTINCT FOREACH/GENERATE MAPREDUCE STREAM SAMPLE
JOIN COGROUP GROUP CROSS CUBE
ORDER RANK LIMIT
UNION SPLIT
DESCRIBE EXPLAIN ILLUSTRATE
REGISTER DEFINE IMPORT #UDF's
AVG CONCAT COUNT MAX MIN SIZE SUM

# Exercise 1
find movies that have an average rating of over 4 stars and order them by release time. 

In [2]:
ratings = LOAD '/user/maria_dev/ml-100k/u.data' AS (userID:int, movieID:int, rating:int, ratingTime:int);
metadata = LOAD '/user/maria_dev/ml-100k/u.item' USING PigStorage('|')
    AS (movieID:int, movieTitle:chararray, releaseDate:chararray, videoRealese:chararray, imdblink:chararray);
#import data and identify delimiters   

nameLookup = FOREACH metadata GENERATE movieID, movieTitle,
    ToUnixTime(ToDate(releaseDate, 'dd-MMM-yyyy')) AS releaseTime;
#restructure the data and convert time

ratingsByMovie = GROUP ratings BY movieID;
avgRatings = FOREACH ratingsByMovie GENERATE group as movieID, AVG(ratings.rating) as avgRating;
#group by movie id and find the average ratings

fiveStarMovies = FILTER avgRatings BY avgRating > 4.0;
#apply filter 

fiveStarsWithData = JOIN fiveStarMovies BY movieID, nameLookup BY movieID;
#join with the name 

oldestFiveStarMovies = ORDER fiveStarsWithData BY nameLookup::releaseTime;
#order by release time 

DUMP oldestFiveStarMovies;


SyntaxError: invalid syntax (<ipython-input-2-5632f0f09fed>, line 1)

# Exercise 2
find old movies that have less than 2 stars
rank them by the number of reviews

In [ ]:
ratings = LOAD '/user/maria_dev/ml-100k/u.data' AS (userID:int, movieID:int, rating:int, ratingTime:int);
metadata = LOAD '/user/maria_dev/ml-100k/u.item' USING PigStorage('|')
    AS (movieID:int, movieTitle:chararray, releaseDate:chararray, videoRealese:chararray, imdblink:chararray);
#import data and identify delimiters   

nameLookup = FOREACH metadata GENERATE movieID, movieTitle,
    ToUnixTime(ToDate(releaseDate, 'dd-MMM-yyyy')) AS releaseTime;
#restructure the data and convert time

ratingsByMovie = GROUP ratings BY movieID;
avgRatings = FOREACH ratingsByMovie GENERATE group as movieID, AVG(ratings.rating) as avgRating,
            COUNT(ratings.rating) as reviews;
#group by movie id and find the average ratings and the count of reviews

fiveStarMovies = FILTER avgRatings BY avgRating < 2.0;
fiveStarsWithData = JOIN fiveStarMovies BY movieID, nameLookup BY movieID;

oldestFiveStarMovies = ORDER fiveStarsWithData BY COUNT(movieID::reviews）;
DUMP oldestFiveStarMovies;